# Shortest Path
Given an integer n representing nodes labeled from 0 to n - 1 in an undirected graph, and an array of non-negative weighted edges, return an array where each index i contains the shortest path length from a specified start node to node i. If a node is unreachable, set its distance to -1.

Each edge is represented by a triplet of positive integers: the start node, the end node, and the weight of the edge.

**Example:**
```python
Input: n = 6,
       edges = [
         [0, 1, 5],
         [0, 2, 3],
         [1, 2, 1],
         [1, 3, 4],
         [2, 3, 4],
         [2, 4, 5],
       ],
       start = 0

Output: [0, 4, 3, 7, 8,-1]
```

## Intuition

There are a few algorithms that can be employed to find the shortest path in a graph. Let’s consider some options:

- **Breadth-First Search (BFS)**: Works well for finding the shortest path when the graph has edges with no weight, or all edges have the same weight. BFS doesn’t take edge weights into account.
- **Dijkstra’s Algorithm**: Works efficiently for graphs with non-negative weights. It finds the shortest path from a single source node to all other nodes using a greedy approach.
- **Bellman-Ford Algorithm**: Useful for graphs with edges that may have negative weights.
- **Floyd-Warshall Algorithm**: Best suited when we need to find the shortest paths between all pairs of nodes in a graph.

Among these, **Dijkstra’s algorithm** is most suitable for our problem since we're dealing with a graph with **non-negative weighted edges**, and we need to find the shortest path from a **start node to all other nodes**.

The greedy choice in Dijkstra’s algorithm becomes evident:  
> At each step, we move to the **unvisited node with the shortest known distance** from the start node.

Applying this logic repeatedly allows Dijkstra’s algorithm to populate an array with the shortest path lengths from the start node to every other node.  
Nodes that are unreachable will have a final distance of **infinity**, which we usually convert to `-1`.

To implement this efficiently, we use a **min-heap** (priority queue), which allows us to quickly retrieve the unvisited node with the smallest distance.

---

### Example of Dijkstra's Algorithm

```python
graph = {
    0: [(1, 2), (2, 4)],
    1: [(3, 3)],
    2: [(3, 1)],
    3: []
}
start_node = 0
```

Let's apply Dijkstra's algorithm from node `0`:

1. Initialize distances: `[0, ∞, ∞, ∞]` (distance to self is 0)
2. Add `(0, 0)` to the min-heap (node 0 with distance 0)
3. Pop `(0, 0)`, visit neighbors:
   - Node 1: 0 + 2 = 2 → update distances to `[0, 2, ∞, ∞]`
   - Node 2: 0 + 4 = 4 → update distances to `[0, 2, 4, ∞]`
4. Pop `(1, 2)`, visit neighbors:
   - Node 3: 2 + 3 = 5 → update distances to `[0, 2, 4, 5]`
5. Pop `(2, 4)` (no better paths found)
6. Pop `(3, 5)` (no neighbors)

Final result: `[0, 2, 4, 5]`

---

### Using the Min-Heap

To understand how the min-heap supports the algorithm:

- The heap always contains nodes prioritized by their **known shortest distance**.
- When popping a node, we only consider processing it if its distance is **still the shortest** found so far.

```python
if curr_dist > distances[curr_node]:
    continue
```

This simple check ensures we avoid reprocessing nodes with outdated distances, eliminating the need for a separate `visited` set.

---

### Why Does the Greedy Approach Work?
Dijkstra's algorithm is greedy because, at every step, it selects the unvisited node with the shortest known distance from the start. It assumes that this is the optimal path to that node — a local optimum — and that this will lead to a global optimum (shortest paths to all nodes).

But can we always guarantee this assumption is correct?

Yes, if all edge weights are non-negative.

Why? Because the only way to find a shorter path to a node already visited would be to go through another node with a smaller total distance — but that’s already been processed. Any other route would require adding at least a positive weight, which can't improve the known minimum.

This is also why Dijkstra's algorithm doesn't work with negative edge weights: a shorter path might exist later, violating the greedy assumption.

In [1]:
from typing import List
from collections import defaultdict
import heapq

def shortest_path(n: int, edges: List[int], start: int) -> List[int]:
    graph = defaultdict(list)
    distances = [float('inf')] * n
    distances[start] = 0

    for u, v, w in edges:
        graph[u].append((v, w))
        graph[v].append((u, w))
    
    min_heap = [(0, start)]

    while min_heap:
        curr_dist, curr_node = heapq.heappop(min_heap)

        if curr_dist > distances[curr_node]:
            continue

        for neighbor, weight in graph[curr_node]:
            neighbor_dist = curr_dist + weight

            if neighbor_dist < distances[neighbor]:
                distances[neighbor] = neighbor_dist
                heapq.heappush(min_heap, (neighbor_dist, neighbor))

    return [-1 if dist == float('inf') else dist for dist in distances]

### Complexity Analysis

#### Time Complexity

Let:
- n be the number of nodes (vertices)
- e  be the number of edges

The time complexity of the `shortest_path` function using **Dijkstra’s algorithm** is: O((n + e) log n)

#### Why:

1. **Creating the adjacency list**:  
   This takes O(e) time, assuming the graph is represented as an edge list initially.

2. **Dijkstra’s algorithm**:
   - Each node is inserted into and removed from the **min-heap** at most once.
   - Each heap operation (`push` and `pop`) takes O(log n) time.
   - For each node, we examine its neighbors; over the whole graph, we process all $$e$$ edges.

   Therefore, Dijkstra's part contributes: O((n + e) log n)

3. **Total time complexity**: O(e) + O((n + e) log n) = O((n + e) log n)

---

#### Space Complexity

The space complexity is: O(n + e)


#### Why:

- O(e) for the **adjacency list**.
- O(n) for:
  - The **distance array** storing shortest distances.
  - The **min-heap** which, in the worst case, contains all n nodes.

So total space usage is: O(n + e)